In [1]:
import sqlparse
from sqlparse.sql import Token, TokenList
import re
import json

In [2]:
# def clean_query(tokens):
#     tokens_flagged = []
#     tokens_exp = []
#     for t in tokens:
#         if isinstance(t,sqlparse.sql.Where):
#             for tt in t.tokens:
#                 tokens_exp.append(tt)
#         else:
#             tokens_exp.append(t)
            
# #     print(tokens_exp)
    
#     for t in tokens_exp:
#         for pattern in ['Token.Text.Whitespace*','Token.Punctuation*']:
#             if re.match(pattern,str(t.ttype)):
#                 tokens_flagged.append(t)
                
#     tokens_cleaned = [t for t in tokens_exp if t not in tokens_flagged]
#     tokens_cleaned = [t for t in tokens_cleaned if type(t) not in [sqlparse.sql.Comment]]
#     return tokens_cleaned

def clean_query(tokens):
    tokens_exp = []
    tokens_cleaned = []
    for t in tokens:
        if isinstance(t,sqlparse.sql.Where):
            for tt in t.tokens:
                tokens_exp.append(tt)
        else:
            tokens_exp.append(t)
            
#     print(tokens_exp)
    
    for t in tokens_exp:
        append_flag = True
        for pattern in ['Token.Text.Whitespace*','Token.Punctuation*']:
            if re.match(pattern,str(t.ttype)):
                append_flag = False
        for t_type in [sqlparse.sql.Comment]:
            if type(t)==t_type:
                append_flag = False
                
        if append_flag:
            tokens_cleaned.append(t)
                
#     tokens_cleaned = [t for t in tokens_exp if t not in tokens_flagged]
#     tokens_cleaned = [t for t in tokens_cleaned if type(t) not in [sqlparse.sql.Comment]]
    return tokens_cleaned

In [3]:
def get_select_from_with(subq_token):
    for t in subq_token.tokens:
        if isinstance(t,sqlparse.sql.Parenthesis):
            return clean_query(t.tokens)

In [4]:
query1 = '''--asd
            with a as (select a_id1 as a_id, --abc
            case when aa = \'a1\' and a2=3 then 1 else 0 end aa_ind --def
            from `a123.aaa`) --pqr
        , b as (select b_id, 
            sum(bb) bb_sum 
            from `b123.bbb` 
            group by b_id) 
        select a.a_id,a.a_ind,b.b_sum 
        from a left join b 
        on a.a_id = b.b_id'''

query2 = '''select a.a_id,a.a_ind,b.b_sum 
        from a left join b 
        on a.a_id = b.b_id'''
query3 = '''
        select a.a_id, --aaa
        max(a.a_ind) a_max,
        sum(b.b_ind) b_sum
        from a left join b 
        on a.a_id = b.b_id
        where a.a_id != 0
        group by 1
        '''
query4 = '''select a,b from aa'''
query5 = '''select a,b from (select aa a,bb b from c)'''
query6 = '''select a_id1 as a_id, --abc
            case when aa = \'a1\' and a2=3 then 1 else 0 end aa_ind --def
            from `a123.aaa`'''

query7 = '''select b_id, 
            sum(bb) bb_sum 
            from `b123.bbb` 
            group by b_id'''

In [5]:
base_keywords = ['select','from','where','group']

In [6]:
# q_c[0].ttype

In [7]:
q_c = clean_query(sqlparse.parse(query3)[0].tokens)
# q_c

In [8]:
def check_token_is_base_keyword(t,kw):
    if kw == 'select':
        return t.match(sqlparse.tokens.Keyword.DML,[kw])
    else:
        return t.match(sqlparse.tokens.Keyword,[kw])
    
# q_c[0].match(sqlparse.tokens.Keyword.DML,['select']),\
# q_c[2].match(sqlparse.tokens.Keyword,['from']),\
# isinstance(q_c[8],sqlparse.sql.Where),\
# q_c[9].match(sqlparse.tokens.Keyword,['group'])

check_token_is_base_keyword(q_c[0],'select'),\
check_token_is_base_keyword(q_c[2],'from'),\
check_token_is_base_keyword(q_c[8],'where'),\
check_token_is_base_keyword(q_c[10],'group')

(True, False, False, False)

In [18]:
def clean_col_def(c,n):
    return re.sub(n+'$','',c.split('--')[0].strip()).strip()
    
    
# def parse_select_statement(s_tokens):
#     op_dict = {}
#     for t in s_tokens:
#         if isinstance(t,sqlparse.sql.Identifier):
#             op_dict[t.get_name()] = clean_col_def(t.value,t.get_alias())
#         if isinstance(t,sqlparse.sql.IdentifierList):
#             for tt in clean_query(t.tokens):
#                 op_dict[tt.get_name()] = clean_col_def(tt.value,tt.get_alias())
#     return op_dict

def parse_select_statement(s_tokens):
    op_dict = {}
    for t in s_tokens:
        if isinstance(t,sqlparse.sql.Identifier):
            if isinstance(clean_query(t.tokens)[0],sqlparse.sql.Case):
                op_dict[t.get_name()] = clean_col_def(t.value,t.get_name())
#                 print(t.value,t.get_name(),clean_col_def(t.value,t.get_name()))
            else:
                op_dict[t.get_name()] = t.get_real_name()
        elif isinstance(t,sqlparse.sql.IdentifierList):
            for tt in clean_query(t.tokens):
                op_dict[tt.get_name()] = tt.get_real_name()
    return op_dict
# def parse_

def parse_from_statement(f_tolens):
    op_dict = {}
    
     

In [19]:

def parse_simple_select(q_tokens):
    assert check_token_is_base_keyword(q_tokens[0],'select')
    
    # initial high level parsing
    q_parse_dict_1 = {kw:[] for kw in base_keywords}
    base_keyword_idx = 0
    for t in q_tokens:
        is_kw_flag = False
        for kw in base_keywords:
            if check_token_is_base_keyword(t,kw):
                key = kw
                is_kw_flag = True
#         if check_token_is_base_keyword(t,base_keywords[base_keyword_idx]):
#             key = base_keywords[base_keyword_idx]
#             q_parse_dict_1[key] = []
#             if base_keyword_idx<len(base_keywords)-1:
#                 base_keyword_idx+=1
#             continue
        if not is_kw_flag:
            q_parse_dict_1[key].append(t)
    q_parse_dict_1['select'] = parse_select_statement(q_parse_dict_1['select'])
    return q_parse_dict_1

In [21]:
parse_simple_select(q_c)

{'select': {'a_id': 'a_id', 'a_max': 'max', 'b_sum': 'sum'},
 'from': [<Identifier 'a' at 0x29721B210C0>,
  <Keyword 'left j...' at 0x29721B1A4C8>,
  <Identifier 'b' at 0x29721B21138>,
  <Keyword 'on' at 0x29721B1A648>,
  <Comparison 'a.a_id...' at 0x29721B211B0>],
 'where': [<Comparison 'a.a_id...' at 0x29721B21228>],
 'group': [<Keyword 'by' at 0x29721B1B048>, <Integer '1' at 0x29721B1B108>]}

In [25]:
def parse_select_q(q):
    parsed_q = sqlparse.parse(q)[0] # assumed that q has a single query
    q_tokens = parsed_q.tokens
    q_tokens_c = clean_query(q_tokens)
    
    if q_tokens_c[0].match(sqlparse.tokens.Keyword.CTE,['with']):
        # do something about query with with
        print('with')
        
        # list,dict of all subqueries
        subq_list_raw = clean_query(q_tokens_c[1].tokens)
        subq_dict = {subq.get_name():parse_simple_select(get_select_from_with(subq)) for subq in subq_list_raw}
#         subq_list_c = subq_dict.values()
        
        # parse individual subqueries
#         print(subq_dict)
        # for q in subq_dict, parse_simple_select
        
        if q_tokens_c[2].match(sqlparse.tokens.Keyword.DML,['select']):
            overall_select_parsed = parse_simple_select(q_tokens_c[2:])
        overall_select_parsed['subqueries'] = subq_dict
        
    elif q_tokens_c[0].match(sqlparse.tokens.Keyword.DML,['select']):
        # simpler query, parse it
        print('select')
        overall_select_parsed = parse_simple_select(q_tokens_c)
    else:
        print(q_tokens_c)
        raise Exception('wut')
        
    return overall_select_parsed
    

In [26]:
print(query7)

select b_id, 
            sum(bb) bb_sum 
            from `b123.bbb` 
            group by b_id


In [27]:
parse_select_q(query1)

with


TypeError: Expected text or file-like object, got <class 'list'>

In [15]:
clean_col_def("case when aa = 'a1' and a2=3 then 1 else 0 end aa_ind","aa_ind")

"case when aa = 'a1' and a2=3 then 1 else 0 end"

In [16]:
check_token_is_base_keyword(clean_query(sqlparse.parse(query7)[0].tokens)[4],'group')

True